In [17]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [ ]:
df.reset_index(drop= True).head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


: 

: 

: 

: 

: 

: 

### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

In [4]:
df_county_tip1 = df.groupby(["county", "order_id"]).agg(tip = ("tip", "mean")).reset_index()

In [5]:
df_county_tip2 = df_county_tip1.groupby("county").agg(tip = ("tip", "mean"), orders= ("order_id", "count")).sort_values(by= "tip", ascending= False).reset_index()

In [6]:
df_county_tip2["orders"] = (df_county_tip2.orders / df_county_tip2.orders.sum()) # *10

In [7]:
df_county_tip2.head()

,county,tip,orders
0,San Diego,0.530895,0.003920
1,Santa Barbara,0.507548,0.023030
2,Lassen,0.502494,0.003964
3,Del Norte,0.501239,0.003323
4,Modoc,0.492131,0.015599


In [147]:
(df_county_tip2.loc[df_county_tip2.county == "Glenn"].orders.values[0] * 100).round(3)

5.856

In [160]:
fig1 = make_subplots(cols= 1, rows= 2, shared_xaxes= True, shared_yaxes= False, vertical_spacing= 0)

for county in df_county_tip2.county:

 if df_county_tip2[df_county_tip2.county == county].tip.values >= 0.5:
  color = px.colors.qualitative.G10[4]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}" 
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

 elif 0.4 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.5:
  color= px.colors.sequential.Greys[3]

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

 elif 0.3 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.4:
  color= px.colors.sequential.Greys[2]

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    base= 0,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

 elif 0.2 < df_county_tip2.loc[df_county_tip2.county == county].tip.values < 0.3:
  color= px.colors.sequential.Greys[1]
  text= df_county_tip2.loc[df_county_tip2.county == county].tip.values.round(3)

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].tip,
    name= f"{county}: {(df_county_tip2.loc[df_county_tip2.county == county].tip.values[0] * 100).round(2)}%",
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    text= text,
    base= 0,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 1
  )

  fig1.add_trace(
   go.Bar(
    x= df_county_tip2.loc[df_county_tip2.county == county].county,
    y= df_county_tip2.loc[df_county_tip2.county == county].orders,
    name= f"Orders: {(df_county_tip2.loc[df_county_tip2.county == county].orders.values[0] * 100).round(3)}%",
    base = -df_county_tip2.loc[df_county_tip2.county == county].orders,
    showlegend= True,
    marker= dict(color= color),
    width= 0.8,
    legendgroup= f"{county}"
   ),
   col= 1,
   row= 2
  )

fig1.add_hline(
 y= 0.5,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= 0.5,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0.3,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= 0.3,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.05,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= -0.05,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_hline(
 y= -0.03,
 opacity= 0.3,
 line_dash= "solid",
 annotation_text= -0.03,
 annotation_position= "top right",
 line_color= px.colors.sequential.Greys[7],
 annotation_font_color= px.colors.sequential.Greys[7],
 annotation_font_size= 12,
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_hline(
 y= 0,
 opacity= 1,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= 0,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.add_vline(
 x= -1,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 1
)

fig1.add_hline(
 y= -0.075,
 opacity= 0.8,
 line_dash= "solid",
 line_color= "black",
 layer= "below",
 col= 1,
 row= 2
)

fig1.update_annotations(opacity= 0.8)
fig1.update_layout(title= dict(text= "<b>Regionale Unterschiede im Trinkgeldverhalten</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.935, x=0.475), legend_title_text = "<b>Counties</b>", height= 700)
fig1.update_xaxes(title_text="<b>Counties</b>", ticks= "outside", col= 1, row= 2, showline= True)
fig1.update_xaxes(showline= False, col= 1, row= 1)
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 1, tickvals= [0, 0.1, 0.2, 0.3, 0.4, 0.5], showline= False)
fig1.update_yaxes(title_text="<b>Order Anteil</b>", ticks= "outside", showgrid= True, gridcolor= "grey", col= 1, row= 2, tickvals= [0, -0.01, -0.03, -0.05, -0.07])
fig1.update_layout(plot_bgcolor= "white", xaxis= dict(linecolor= "black"), yaxis= dict(linecolor= "black"))
fig1.show()

In [157]:
# px.colors.qualitative.G10[4]
# px.colors.sequential.Greys[]

#### old

In [ ]:
df_county_tip = df.groupby("county").agg(tip = ("tip", "sum"), tip_opp = ("tip", "count")).reset_index()
df_county_tip["no_tip"] = df_county_tip.tip_opp - df_county_tip.tip

In [ ]:
df_county_tip["tip_norm"], df_county_tip["no_tip_norm"] = [df_county_tip.iloc[:, 1:2].apply(lambda x: x / df_county_tip.tip_opp, axis= 0), df_county_tip.iloc[:, 3:4].apply(lambda x: x / df_county_tip.tip_opp, axis= 0)]

In [ ]:
df_county_tip.drop(["tip_opp"], axis= 1, inplace= True)

In [ ]:
df_county_tip.head()

,county,tip,no_tip,tip_norm,no_tip_norm
0,Alameda,7071,9845,0.418007,0.581993
1,Alpine,59712,73571,0.448009,0.551991
2,Amador,55238,65874,0.456090,0.543910
3,Calaveras,229220,273259,0.456178,0.543822
4,Colusa,16970,17973,0.485648,0.514352


In [ ]:
df_county_tip.loc[df_county_tip.tip_norm > df_county_tip.no_tip_norm]

,county,tip,no_tip,tip_norm,no_tip_norm
6,Del Norte,10075,9235,0.521750,0.478250
7,El Dorado,8183,7240,0.530571,0.469429
10,Humboldt,3321,3080,0.518825,0.481175
16,Lassen,11678,11522,0.503362,0.496638
23,Modoc,50644,44438,0.532635,0.467365
24,Mono,15075,14790,0.504771,0.495229
34,San Bernardino,75,73,0.506757,0.493243
35,San Diego,13636,10684,0.560691,0.439309
40,Santa Barbara,93390,80136,0.538190,0.461810
53,Tuolumne,16847,15186,0.525926,0.474074


In [ ]:
pd.crosstab(df.county, df.tip, margins= True, normalize= False).head()

tip,0,1,All
county,,,
Alameda,9845,7071,16916
Alpine,73571,59712,133283
Amador,65874,55238,121112
Calaveras,273259,229220,502479
Colusa,17973,16970,34943


In [ ]:
df_county_tip_sorted = df_county_tip.sort_values(by= ["tip_norm"], ascending= False)
px.bar(df_county_tip_sorted, x= "county", y= "tip_norm")